In [3]:
pip install yfinance

     |████████████████████████████████| 6.4 MB 16.0 MB/s 
     |████████████████████████████████| 63 kB 1.3 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [4]:
from statsmodels.tsa.seasonal import seasonal_decompose
from dateutil.parser import parse
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.4f' % x)
import seaborn as sns
sns.set_context("paper", font_scale=1.3)
sns.set_style('white')
import warnings
warnings.filterwarnings('ignore')
from time import time
import matplotlib.ticker as tkr
from scipy import stats
from statsmodels.tsa.stattools import adfuller
from sklearn import preprocessing
from statsmodels.tsa.stattools import pacf
%matplotlib inline
import math
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from keras.callbacks import EarlyStopping
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier


import yfinance as yf

In [ ]:
print('category 1:')
category = input()
print('category 2:')
category1 = input()
print('category 3:')
category2 = input()
print('category 4:')
category3 = input()
print('ticker:')  
ticker = input()
print('visual (yes/no):')
descriptive = input()
print('days back (yes/no):')
days_back = input()
print('number of trees:')
num_trees = 10000
print('scrape (yes/no):')
scrape = input()
print('shift days:')
shift = input()
start_date = '2021-09-03'
print('setup (on/off):')
setup = input()
print('shift change (on/off):')
shift_change = input()
print('random forest: (yes/no)')
random_forest = input()
print('xgboost (yes/no):')
xgb = input()
print('logit (yes/no):')
logi_regression = input()
print('gradient boosted (yes/no):')
gradient_boosted = input()

#------------------------------------------------------

if descriptive == 'yes':
  msft = yf.Ticker(ticker.upper())
  df = msft.history(period="max")
  df = df.iloc[7000:]
  df = df.reset_index()

  df['date_time'] = df['Date']
  df = df.dropna()
  df['date_time']=pd.to_datetime(df['date_time']) 
  df['year'] = df['date_time'].apply(lambda x: x.year)
  df['quarter'] = df['date_time'].apply(lambda x: x.quarter)
  df['month'] = df['date_time'].apply(lambda x: x.month)
  df['day'] = df['date_time'].apply(lambda x: x.day)
  df=df.loc[:,['date_time','Close', 'year','quarter','month','day']]
  df.sort_values('date_time', inplace=True, ascending=True)
  df = df.reset_index(drop=True)
  df["weekday"]=df.apply(lambda row: row["date_time"].weekday(),axis=1)
  df["weekday"] = (df["weekday"] < 5).astype(int)
  print('Number of rows and columns after removing missing values:', df.shape)
  print('The time series starts from: ', df.date_time.min())
  print('The time series ends on: ', df.date_time.max())

  stat, p = stats.normaltest(df.Close)
  print('Statistics=%.3f, p=%.3f' % (stat, p))
  alpha = 0.05
  if p > alpha:
      print('Data looks Gaussian (fail to reject H0)')
  else:
      print('Data does not look Gaussian (reject H0)')

  sns.distplot(df.Close);
  print( 'Kurtosis of normal distribution: {}'.format(stats.kurtosis(df.Close)))
  print( 'Skewness of normal distribution: {}'.format(stats.skew(df.Close)))

  df1=df.loc[:,['date_time','Close']]
  df1.set_index('date_time',inplace=True)
  df1.plot(figsize=(12,5))
  plt.ylabel('MSFT')
  plt.legend().set_visible(False)
  plt.tight_layout()
  plt.title('Microsoft Time Series')
  sns.despine(top=True)
  plt.show();

  plt.figure(figsize=(14,5))
  plt.subplot(1,2,1)
  plt.subplots_adjust(wspace=0.2)
  sns.boxplot(x="year", y="Close", data=df)
  plt.xlabel('year')
  plt.title('Box plot of MSFT stock')
  sns.despine(left=True)
  plt.tight_layout()
  plt.subplot(1,2,2)
  sns.boxplot(x="quarter", y="Close", data=df)
  plt.xlabel('quarter')
  plt.title('Box plot of Quarterly Stock Closing Prices')
  sns.despine(left=True)
  plt.tight_layout();

  plt.figure(figsize=(14,6))
  plt.subplot(1,2,1)
  df['Close'].hist(bins=50)
  plt.title('MSFT Closing Price Distribution')
  plt.subplot(1,2,2)
  stats.probplot(df['Close'], plot=plt);
  df1.describe().T

  fig = plt.figure(figsize=(18,16))
  fig.subplots_adjust(hspace=.4)
  ax1 = fig.add_subplot(5,1,1)
  ax1.plot(df1['Close'].resample('D').mean(),linewidth=1)
  ax1.set_title('Mean MSFT Closing Price resampled over day')
  ax1.tick_params(axis='both', which='major')

  ax2 = fig.add_subplot(5,1,2, sharex=ax1)
  ax2.plot(df1['Close'].resample('W').mean(),linewidth=1)
  ax2.set_title('Mean MSFT Closing Price power resampled over week')
  ax2.tick_params(axis='both', which='major')

  ax3 = fig.add_subplot(5,1,3, sharex=ax1)
  ax3.plot(df1['Close'].resample('M').mean(),linewidth=1)
  ax3.set_title('Mean MSFT Closing Price resampled over month')
  ax3.tick_params(axis='both', which='major')

  ax4  = fig.add_subplot(5,1,4, sharex=ax1)
  ax4.plot(df1['Close'].resample('Q').mean(),linewidth=1)
  ax4.set_title('Mean MSFT Closing Price over quarter')
  ax4.tick_params(axis='both', which='major')

  ax5  = fig.add_subplot(5,1,5, sharex=ax1)
  ax5.plot(df1['Close'].resample('A').mean(),linewidth=1)
  ax5.set_title('Mean MSFT Closing Price resampled over year')
  ax5.tick_params(axis='both', which='major');

  plt.figure(figsize=(14,8))
  plt.subplot(2,2,1)
  df.groupby('year').Close.agg('mean').plot()
  plt.xlabel('')
  plt.title('Mean MSFT Closing Price by Year')

  plt.subplot(2,2,2)
  df.groupby('quarter').Close.agg('mean').plot()
  plt.xlabel('')
  plt.title('Mean MSFT Closing Price by Quarter')

  plt.subplot(2,2,3)
  df.groupby('month').Close.agg('mean').plot()
  plt.xlabel('')
  plt.title('Mean MSFT Closing Price by Month')

  plt.subplot(2,2,4)
  df.groupby('day').Close.agg('mean').plot()
  plt.xlabel('')
  plt.title('Mean MSFT Closing Price by Day');


  pd.pivot_table(df.loc[df['year'] != 2013], values = "Close", 
                columns = "year", index = "month").plot(subplots = True, figsize=(12, 12), layout=(3, 5), sharey=True);

  dic={0:'Weekend',1:'Weekday'}
  df['Day'] = df.weekday.map(dic)
  a=plt.figure(figsize=(9,4)) 
  plt1=sns.boxplot('year','Close',hue='Day',width=0.6,fliersize=3,
                      data=df)                                                                                                                                                                                                                                                                                                                                                 
  a.legend(loc='upper center', bbox_to_anchor=(0.5, 1.00), shadow=True, ncol=2)
  sns.despine(left=True, bottom=True) 
  plt.xlabel('')
  plt.tight_layout()                                                                                                                  
  plt.legend().set_visible(False);

  plt1=sns.factorplot('year','Close',hue='Day',
                      data=df, size=4, aspect=1.5, legend=False)                                                                                                                                                                                                                                                                                                                                             
  plt.title('Factor Plot of MSFT Closign Price by Weekend/Weekday')                                                             
  plt.tight_layout()                                                                                                                  
  sns.despine(left=True, bottom=True) 
  plt.legend(loc='upper right');
else:
  print('no visualization')

if scrape == 'yes':
  urls = [
    'https://companiesmarketcap.com/automakers/largest-automakers-by-market-cap/',
    'https://companiesmarketcap.com/automakers/largest-automakers-by-market-cap/',
    'https://companiesmarketcap.com/airlines/largest-airlines-by-market-cap/',
    'https://companiesmarketcap.com/airports/largest-airport-operating-companies-by-market-cap/',
    'https://companiesmarketcap.com/aircraft-manufacturers/largest-aircraft-manufacturers-by-market-cap/',
    'https://companiesmarketcap.com/banks/largest-banks-by-market-cap/',
    'https://companiesmarketcap.com/hotels/largest-hotel-companies-by-market-cap/',
    'https://companiesmarketcap.com/pharmaceuticals/largest-pharmaceutical-companies-by-market-cap/',
    'https://companiesmarketcap.com/e-commerce/largest-e-commerce-companies-by-market-cap/',
    'https://companiesmarketcap.com/healthcare/largest-healthcare-companies-by-market-cap/',
    'https://companiesmarketcap.com/ports/largest-port-operating-companies-by-market-cap/',
    'https://companiesmarketcap.com/professional-services/largest-professional-service-companies-by-market-cap/',
    'https://companiesmarketcap.com/food/largest-food-companies-by-market-cap/',
    'https://companiesmarketcap.com/restaurant-chains/largest-restaurant-chain-companies-by-market-cap/',
    'https://companiesmarketcap.com/software/largest-software-companies-by-market-cap/',
    'https://companiesmarketcap.com/semiconductors/largest-semiconductor-companies-by-market-cap/',
    'https://companiesmarketcap.com/tobacco/largest-tobacco-companies-by-market-cap/',
    'https://companiesmarketcap.com/financial-services/largest-financial-service-companies-by-market-cap/',
    'https://companiesmarketcap.com/oil-gas/largest-oil-and-gas-companies-by-market-cap/',
    'https://companiesmarketcap.com/electricity/largest-electricity-companies-by-market-cap/',
    'https://companiesmarketcap.com/delivery-services/largest-delivery-companies-by-market-cap/',
    'https://companiesmarketcap.com/media-press/largest-media-and-press-companies-by-market-cap/',
    'https://companiesmarketcap.com/alcoholic-beverages/largest-alcoholic-beverage-companies-by-market-cap/',
    'https://companiesmarketcap.com/beverages/largest-beverage-companies-by-market-cap/',
    'https://companiesmarketcap.com/clothing/largest-clothing-companies-by-market-cap/',
    'https://companiesmarketcap.com/mining/largest-mining-companies-by-market-cap/',
    'https://companiesmarketcap.com/railways/largest-railways-companies-by-market-cap/',
    'https://companiesmarketcap.com/insurance/largest-insurance-companies-by-market-cap/',
    'https://companiesmarketcap.com/real-estate/largest-real-estate-companies-by-market-cap/',
    'https://companiesmarketcap.com/chemicals/largest-chemical-companies-by-market-cap/',
    'https://companiesmarketcap.com/investment/largest-investment-companies-by-market-cap/',
    'https://companiesmarketcap.com/telecommunication/largest-telecommunication-companies-by-market-cap/',
    'https://companiesmarketcap.com/retail/largest-retail-companies-by-market-cap/',
    'https://companiesmarketcap.com/internet/largest-internet-companies-by-market-cap/',
    'https://companiesmarketcap.com/construction/largest-construction-companies-by-market-cap/',
    'https://companiesmarketcap.com/tech/largest-tech-companies-by-market-cap/'
  ]
  a = []
  for x in range(len(urls)):
      a.append(urls[x].split('/')[3])
  a

  import requests
  import time
  import bs4
  from bs4 import BeautifulSoup
  cats = ['automakers','airlines','aircraft-manufacturers','banks','pharmaceuticals','e-commerce','healthcare','ports','professional-services','food','restaurant-chains','software','semiconductors','tobacco','financial-services','electricity','courier-services','media/press','alcoholic-beverages','beverages','clothing','mining','railways','insurance','real-estate','chemicals','investment','telecomunication','retail','internet','construction','tech']
  cat_names = {'category':[],'names':[],'ticker':[]}
  try:
      for x in range(len(urls)):
          url = urls[x]
          html = requests.get(url).text
          soup = bs4.BeautifulSoup(html, "html.parser")
          for tag in soup.findAll("div", {"class": "name-div"}):
                  cat_names['names'].append(tag.findNext("div").text)
                  cat_names['category'].append(a[x])
          for tag in soup.findAll("div", {"class": "company-name"}):
                  cat_names['ticker'].append(tag.findNext().text)

  except:
      print(x)
else:
  print('no scrape')

if setup == 'on':
  cat_names = pd.DataFrame(cat_names)
  tech = cat_names['ticker'][cat_names['category']==category].append(cat_names['ticker'][cat_names['category']==category1] ).append(cat_names['ticker'][cat_names['category']==category2] ).append(cat_names['ticker'][cat_names['category']==category3] )
  tech = tech.reset_index()
  tech = tech.drop(columns = 'index')
  tech = tech['ticker']

  for x in range(len(tech)):
    try:
        xx  = yf.Ticker(tech.iloc[x])
        df = xx.history(period="max")
        df.columns.values[0] = tech.iloc[x]+" Open"
        df.columns.values[1] = tech.iloc[x]+" High"
        df.columns.values[2] = tech.iloc[x]+" Low"
        df.columns.values[3] = tech.iloc[x]+" Close"
        df.columns.values[4] = tech.iloc[x]+" Volume"
        df.columns.values[5] = tech.iloc[x]+" Dividends"
        df.columns.values[6] = tech.iloc[x]+" Stock Splits"
        tech.iloc[x] = df.reset_index()
    except:
      print(tech.iloc[x])
  

  from sklearn import preprocessing
  from sklearn.model_selection import train_test_split
  from sklearn.ensemble import RandomForestClassifier
  from sklearn import metrics
  import time
  label_encoder = preprocessing.LabelEncoder()

  msft = yf.Ticker(ticker)
  df = msft.history(period="max")

  for ii in range(len(tech)):
    if len(tech.iloc[ii])>500:
      df = pd.merge(df, tech.iloc[ii] ,on = 'Date')
    else:
      print('no ')
else:
      print('no setup')

if shift_change == 'on':
  tech = cat_names['ticker'][cat_names['category']==category].append(cat_names['ticker'][cat_names['category']=='financial-services'] ).append(cat_names['ticker'][cat_names['category']=='internet'] ).append(cat_names['ticker'][cat_names['category']=='e-commerce'] )
  tech = tech.reset_index()
  tech = tech.drop(columns = 'index')
  tech = tech['ticker']
  aa = pd.DataFrame()
  vol = pd.DataFrame()
  ss = []
  for i in range(len(tech)):
    try:
      aa[str(tech.iloc[i])] = (df[str(tech.iloc[i])+' Close']- df[str(tech.iloc[i])+' Open'])/df[str(tech.iloc[i])+' Open']*100
      vol[str(tech.iloc[i])] = df[str(tech.iloc[i])+' Volume']
    except:
      ss.append(str(tech.iloc[i]))
  aa = round(aa)

  tech = tech.drop(columns = [ss])
  tech = [item for item in tech if item.isalpha()]
  tech = pd.DataFrame(tech)
  dd = pd.DataFrame()

  rr = pd.DataFrame()
  for x in range(len(tech)):
    try:
      rr[tech.iloc[x]] = aa[tech.iloc[x]]
    except:
      print(tech.iloc[x])

  qq = pd.DataFrame()
  for x in range(len(tech)):
    try:
      qq[tech.iloc[x]] = aa[tech.iloc[x]]
    except:
      print(tech.iloc[x])



  ss = []
  df = pd.DataFrame()

  for x in range(len(tech)):
    try:
      df = qq.drop(columns = tech[0][x])
      ss.append(df)
    except:
      print(tech[0][x])


  pieces = []
  for x in range(len(tech)):
    try:
      pieces.append(ss[x].iloc[-1*days_back:])
    except:
      print(tech[0][x])
  df_final = pd.concat(pieces,axis = False, ignore_index = True)


  tt = df_final
  for x in range(len(tech)):
    try:
      tt[tech[0][x]].fillna(value=tt[tech[0][x]].mean(), inplace=True)
    except:
      print(tech[0][x])

  y = []
  for x in range(len(tech)):
    try:
      y.append(qq[tech[0][x]].iloc[-1*days_back:])
    except:
      print(tech[0][x])
  yy = pd.concat(y, ignore_index = True)

  tt = round(tt)

  x = tt.shift(shift*-1)
  y = yy.shift(shift)
  x = tt.dropna()
  xx = x[shift:]
  xx = xx.reset_index()
  y = y.dropna()
  y = y.loc[~(y==0)]
  yy = pd.DataFrame()
  yy['y'] = y
  yy = yy.reset_index()
  xxx = pd.merge(yy,xx, on = 'index')
  y = xxx['y']
  xxx = xxx.drop(columns = ['index','y'])

  x = xxx
  xx = x

  yy = y

  xxx = x
  yyy = y
else:
  print('no setup')

if random_forest == 'yes':
  X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
  clf=RandomForestClassifier(n_estimators=num_trees)
  clf.fit(X_train,y_train)
  y_pred=clf.predict(X_test)
  print('random forest score = '+str(metrics.accuracy_score(y_test, y_pred)))
else:
  print('no random forest')

if logi_regression == 'yes':
  xx, yy = make_classification(n_samples=1000, n_features=10, n_informative=5, n_redundant=5, n_classes=3, random_state=1)

  model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
  cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
  n_scores = cross_val_score(model, x, y, scoring='accuracy', cv=cv, n_jobs=-1)

  print('Logistic mean accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))
else:
  print('no logistic regression')

if gradient_boosted == 'yes':
  X_train, X_test, y_train, y_test = train_test_split(xxx, yyy, test_size=0.2)

  lr_list = [0.05, 0.075, 0.1, 0.25, 0.5, 0.75, 1]

  for learning_rate in lr_list:
      gb_clf = GradientBoostingClassifier(n_estimators=20, learning_rate=learning_rate, max_features=2, max_depth=2, random_state=0)
      gb_clf.fit(X_train, y_train)

      print("Learning rate: ", learning_rate)
      print("Gradient Boosted Accuracy score (training): {0:.3f}".format(gb_clf.score(X_train, y_train)))
      print("Gradient Boosted Accuracy score (validation): {0:.3f}".format(gb_clf.score(X_test, y_test)))
else:
  print('no')


if xgb == 'yes':
  xgb_clf = XGBClassifier()
  xgb_clf.fit(X_train, y_train)
  score = xgb_clf.score(X_test, y_test)
  print('XGB Score = '+str(score))
else:
  print('no')



category 1:
tech
category 2:
internet
category 3:
financial-services
category 4:
e-commerce
ticker:
MSFT
visual (yes/no):
no
input (yes/no):
30
number of trees:
scrape (yes/no):
no
shift days:
1
setup (on/off):
on
shift change (on/off):
on
random forest: (yes/no)
yes
xgboost (yes/no):
yes
logit (yes/no):
yes
gradient boosted (yes/no):
yes
no visualization
no scrape
- ETH: No data found, symbol may be delisted
ETH
- USDT: No data found, symbol may be delisted
USDT


In [6]:
xxx

,MSFT,AMZN,TSLA,NVDA,TSM,ASML,BABA,AVGO,CSCO,ORCL,...,JMIA,REAL,LQDT,PETS,BZUN,GRPN,ECOM,PRTS,NHTC,AAPL
0,-1.0000,-1.0000,5.0000,-1.0000,-0.0000,-2.0000,1.0000,-0.0000,-2.0000,-5.0000,...,-5.0000,-5.0000,22.0000,4.0000,7.0000,-1.0000,3.0000,3.0000,1.0000,-0.0000
1,-1.0000,-1.0000,-4.0000,-5.0000,-1.0000,-2.0000,-1.0000,-2.0000,0.0000,5.0000,...,-7.0000,-2.0000,-4.0000,0.0000,3.0000,-4.0000,5.0000,-3.0000,1.0000,-0.0000
2,-4.0000,-0.0000,0.0000,-7.0000,-3.0000,-3.0000,-1.0000,-3.0000,-5.0000,-3.0000,...,-10.0000,5.0000,-6.0000,-10.0000,-3.0000,-4.0000,-4.0000,1.0000,7.0000,-0.0000
3,-0.0000,1.0000,-1.0000,1.0000,0.0000,0.0000,-1.0000,2.0000,1.0000,2.0000,...,3.0000,18.0000,3.0000,4.0000,1.0000,2.0000,5.0000,-3.0000,4.0000,-0.0000
4,2.0000,3.0000,4.0000,3.0000,1.0000,1.0000,1.0000,-1.0000,-1.0000,1.0000,...,-4.0000,6.0000,6.0000,2.0000,3.0000,4.0000,-0.0000,13.0000,-0.0000,-0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1504,-0.0000,1.0000,-1.0000,1.0000,0.0000,0.0000,-1.0000,2.0000,1.0000,2.0000,...,3.0000,18.0000,3.0000,4.0000,1.0000,2.0000,5.0000,-3.0000,2.0000,1.0000
1505,2.0000,3.0000,4.0000,3.0000,1.0000,1.0000,1.0000,-1.0000,-1.0000,1.0000,...,-4.0000,6.0000,6.0000,2.0000,3.0000,4.0000,-0.0000,13.0000,2.0000,-0.0000
1506,1.0000,-0.0000,4.0000,3.0000,1.0000,2.0000,-1.0000,2.0000,1.0000,0.0000,...,6.0000,0.0000,-5.0000,-1.0000,2.0000,9.0000,-3.0000,8.0000,2.0000,1.0000
1507,-1.0000,-3.0000,8.0000,1.0000,1.0000,-1.0000,-2.0000,2.0000,1.0000,0.0000,...,2.0000,13.0000,10.0000,10.0000,5.0000,11.0000,6.0000,12.0000,2.0000,0.0000


In [ ]:
yy

array([25, 26, 30, ..., 28, 33, 13])

In [ ]:
if logi_regression == 'yes':
  xx, yy = make_classification(n_samples=100000, n_features=int(len(x.keys())), n_informative=int(len(y.unique())), n_redundant=5, n_classes=int(len(y.unique())), random_state=1)

  model = LogisticRegression(multi_class='multinomial', solver='lbfgs').fit(xx,yy)
  cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
  n_scores = cross_val_score(model, x, y, scoring='accuracy', cv=cv, n_jobs=-1)

  print('Logistic mean accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))


Logistic mean accuracy: 0.199 (0.025)


In [ ]:
pd.DataFrame(model.predict_proba(x.iloc[-10:]))

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,33
0,0.0262,0.0277,0.0049,0.0066,0.0042,0.0157,0.0307,0.0176,0.0087,0.0108,...,0.0075,0.0088,0.0269,0.0325,0.1047,0.0564,0.0021,0.0301,0.0959,0.0075
1,0.0026,0.0279,0.1800,0.0897,0.0069,0.0049,0.0302,0.0006,0.0109,0.0071,...,0.0027,0.0451,0.1525,0.0591,0.0022,0.0083,0.1291,0.0070,0.0136,0.0394
2,0.0358,0.0007,0.0080,0.0015,0.0446,0.0065,0.0692,0.0046,0.0081,0.0057,...,0.0084,0.0368,0.0415,0.0110,0.1207,0.0009,0.0022,0.0351,0.0126,0.0024
3,0.0766,0.0020,0.1917,0.0116,0.0518,0.0627,0.0109,0.0119,0.0085,0.0047,...,0.0161,0.0009,0.0162,0.0176,0.0974,0.0158,0.0021,0.0125,0.0092,0.0077
4,0.0145,0.0113,0.0278,0.0004,0.0021,0.0497,0.0193,0.0828,0.0067,0.0007,...,0.0011,0.0018,0.0082,0.0063,0.1017,0.0271,0.0034,0.0746,0.0663,0.0110
5,0.0338,0.0344,0.0029,0.0190,0.0044,0.0138,0.0018,0.0182,0.0267,0.0465,...,0.0119,0.0311,0.0349,0.0157,0.0014,0.0055,0.1808,0.0102,0.0020,0.0293
6,0.1088,0.0458,0.0178,0.0003,0.0012,0.0013,0.0518,0.0392,0.0033,0.0014,...,0.0283,0.0081,0.0099,0.0283,0.3284,0.0010,0.0055,0.0131,0.0024,0.0078
7,0.0073,0.0408,0.0268,0.0049,0.0014,0.0039,0.1505,0.0154,0.0043,0.0129,...,0.0153,0.0279,0.0144,0.0049,0.0433,0.0032,0.1188,0.0190,0.0190,0.0045
8,0.0031,0.0083,0.0056,0.0164,0.0031,0.0081,0.0056,0.0309,0.0100,0.0153,...,0.0647,0.0427,0.0206,0.0552,0.0828,0.0013,0.0135,0.0091,0.0003,0.0074
9,0.0260,0.0311,0.0050,0.0044,0.0027,0.0108,0.0429,0.0206,0.0087,0.0094,...,0.0089,0.0100,0.0273,0.0375,0.1411,0.0313,0.0017,0.0383,0.0858,0.0058


array([[1.08800997e-02, 9.61201773e-01, 2.79181273e-02],
       [2.49700295e-02, 8.61618957e-07, 9.75029109e-01],
       [5.03148600e-01, 4.96790264e-01, 6.11359926e-05],
       [8.31508831e-02, 2.31910531e-02, 8.93658064e-01],
       [9.72698381e-01, 4.77376426e-06, 2.72968449e-02],
       [2.95923744e-06, 2.42978555e-04, 9.99754062e-01],
       [4.98465394e-03, 9.94096188e-01, 9.19157773e-04],
       [7.18958211e-01, 2.79647844e-01, 1.39394565e-03],
       [1.81837051e-06, 9.99652102e-01, 3.46079355e-04],
       [6.28576811e-03, 9.90415446e-01, 3.29878591e-03]])

array([ 2., -3., -2., ...,  1., 10.,  4.])

In [ ]:
df_compare = pd.DataFrame()
df_compare['prediction'] = list(clf.predict(xxx))
df_compare['actual'] = y_test.reset_index().drop(columns = 'index')
df_compare.head(50)

,prediction,actual
0,2.0000,-3.0000
1,-1.0000,1.0000
2,-1.0000,1.0000
3,-4.0000,1.0000
4,2.0000,2.0000
5,-1.0000,-3.0000
6,-3.0000,4.0000
7,5.0000,1.0000
8,-1.0000,-1.0000
9,-1.0000,-3.0000


In [ ]:

model.predict_proba(xxx)


NotFittedError: ignored

In [ ]:
y

1      0.0000
2      0.0000
3      0.0000
4     -0.0000
5     -0.0000
        ...  
835   -0.0000
836   -0.0000
837    0.0000
838    0.0000
839   -0.0000
Length: 838, dtype: float64

In [ ]:
  y = []
  for x in range(len(tech)):
    try:
      y.append(qq[tech[0][x]])
    except:
      print(tech[0][x])
  yy = pd.concat(y, ignore_index = True)

  tt = round(tt)

  x = tt.shift(shift*-1)
  y = yy.shift(shift)

  x = tt.dropna()
  x = x[1:]
  y = y.dropna()


ABNB
SNOW
XIACF
COIN
GFS
CPNG


In [ ]:
y

1        -0.0000
2        -1.0000
3        -0.0000
4        -1.0000
5         1.0000
          ...   
46830    -6.0000
46831   -12.0000
46832     0.0000
46833     1.0000
46834   -10.0000
Length: 46834, dtype: float64

In [ ]:
x[2:]

,MSFT,GOOG,AMZN,TSLA,FB,NVDA,TSM,TCEHY,ASML,AVGO,...,EA,PDD,EBAY,ALGN,ERIC,ZM,EXPE,CERN,NET,AAPL
2,-0.0000,8.0000,8.0000,0.0000,-3.0000,1.0000,-0.0000,0.0000,2.0000,2.0000,...,-0.0000,-0.0000,-0.0000,-2.0000,0.0000,0.0000,-1.0000,0.0000,1.0000,0.0000
3,-3.0000,-21.0000,-49.0000,-4.0000,-7.0000,-1.0000,-0.0000,-0.0000,-2.0000,-5.0000,...,-1.0000,-2.0000,-1.0000,-2.0000,-0.0000,-2.0000,-1.0000,-0.0000,-1.0000,0.0000
4,2.0000,31.0000,21.0000,1.0000,1.0000,1.0000,1.0000,0.0000,5.0000,4.0000,...,1.0000,1.0000,0.0000,-1.0000,0.0000,0.0000,1.0000,0.0000,-0.0000,0.0000
5,0.0000,-1.0000,-23.0000,2.0000,-1.0000,-0.0000,1.0000,-0.0000,-1.0000,2.0000,...,-1.0000,0.0000,-0.0000,-4.0000,-0.0000,0.0000,-0.0000,-0.0000,-1.0000,0.0000
6,-2.0000,-18.0000,-23.0000,-0.0000,-3.0000,-1.0000,-0.0000,-1.0000,-3.0000,-2.0000,...,-1.0000,-2.0000,-0.0000,-4.0000,0.0000,-4.0000,-1.0000,-0.0000,-1.0000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46830,-10.0000,-109.0000,-160.0000,-52.0000,-13.0000,-15.0000,-4.0000,-2.0000,-43.0000,-26.0000,...,-0.0000,-1.0000,-1.0000,-28.0000,-0.0000,-3.0000,-20.0000,-0.0000,0.0000,-4.0000
46831,-2.0000,21.0000,-13.0000,29.0000,2.0000,2.0000,-0.0000,-0.0000,-1.0000,4.0000,...,-3.0000,2.0000,-2.0000,-15.0000,-0.0000,2.0000,7.0000,-0.0000,0.0000,-1.0000
46832,3.0000,24.0000,23.0000,-13.0000,0.0000,1.0000,-0.0000,-1.0000,1.0000,2.0000,...,-1.0000,-4.0000,-1.0000,2.0000,0.0000,-3.0000,9.0000,-0.0000,0.0000,-2.0000
46833,-8.0000,-70.0000,-81.0000,-45.0000,-5.0000,-9.0000,-4.0000,-2.0000,-28.0000,-14.0000,...,-2.0000,-5.0000,-3.0000,-37.0000,-0.0000,-5.0000,-2.0000,0.0000,0.0000,-4.0000


In [ ]:
y

2        -0.0000
3        -1.0000
4        -0.0000
5        -1.0000
6         1.0000
          ...   
46830   -14.0000
46831    -6.0000
46832   -12.0000
46833     0.0000
46834     1.0000
Length: 46833, dtype: float64